# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [16]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import six
from sklearn.model_selection import train_test_split

from tensorflow.python.client import device_lib

device_lib.list_local_devices()

# tf.__version__sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# print("VERSION", tf.__version__sess)

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17666686349556247211, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 104923136
 locality {
   bus_id: 1
 }
 incarnation: 14157992577450473724
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0"]

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
# sys.path.append("..")
sys.path.append("./object_detection/") # Pavlo's machine
# sys.path.append("../../../../../../tensorflow-models/models/research/object_detection/") # Pavlo's machine

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

from utils import dataset_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# Change this to the dir
DATA_DIR = '../../../../../../data_capstone'

# What model to download.

# MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_NAME = 'faster_rcnn_resnet101_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_CKPT = os.path.join(DATA_DIR, MODEL_NAME + '/frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('./object_detection', 'data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

In [5]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:
print(category_index)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic light'}, 11: {'id': 11, 'name': 'fire hydrant'}, 13: {'id': 13, 'name': 'stop sign'}, 14: {'id': 14, 'name': 'parking meter'}, 15: {'id': 15, 'name': 'bench'}, 16: {'id': 16, 'name': 'bird'}, 17: {'id': 17, 'name': 'cat'}, 18: {'id': 18, 'name': 'dog'}, 19: {'id': 19, 'name': 'horse'}, 20: {'id': 20, 'name': 'sheep'}, 21: {'id': 21, 'name': 'cow'}, 22: {'id': 22, 'name': 'elephant'}, 23: {'id': 23, 'name': 'bear'}, 24: {'id': 24, 'name': 'zebra'}, 25: {'id': 25, 'name': 'giraffe'}, 27: {'id': 27, 'name': 'backpack'}, 28: {'id': 28, 'name': 'umbrella'}, 31: {'id': 31, 'name': 'handbag'}, 32: {'id': 32, 'name': 'tie'}, 33: {'id': 33, 'name': 'suitcase'}, 34: {'id': 34, 'name'

## Helper code

In [9]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [10]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.

# PATH_TO_TEST_IMAGES_DIR = os.path.join(DATA_DIR, 'test_images')
# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'just_traffic_light_0{}.jpg'.format(120 + i)) for i in range(0, 3) ]

PATH_TO_TEST_IMAGES_DIR = os.path.join(DATA_DIR, 'sim_test_images')
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'session1_{:02d}.jpg'.format(i + 1)) for i in range(0, 5) ]

print(TEST_IMAGE_PATHS)


# Size, in inches, of the output images.
IMAGE_SIZE = (16, 12)

['../../../../../../data_capstone/sim_test_images/session1_01.jpg', '../../../../../../data_capstone/sim_test_images/session1_02.jpg', '../../../../../../data_capstone/sim_test_images/session1_03.jpg', '../../../../../../data_capstone/sim_test_images/session1_04.jpg', '../../../../../../data_capstone/sim_test_images/session1_05.jpg']


In [11]:
def encode_image_array_as_jpg_str(image):
  """Encodes a numpy array into a JPEG string.

  Args:
    image: a numpy array with shape [height, width, 3].

  Returns:
    JPEG encoded image string.
  """
  image_pil = Image.fromarray(np.uint8(image))
  output = six.BytesIO()
  image_pil.save(output, format='JPEG')
  jpg_string = output.getvalue()
  output.close()
  return jpg_string

def create_record(box, newclass, image, filepath):
    textlabel = [ 'RED', 'YELLOW', 'GREEN' ]
    label = [ 1, 2, 3 ]
    
    height = image.shape[0]
    width = image.shape[1]
    filename = filepath
    encoded_image_data = encode_image_array_as_jpg_str(image)
    image_format = b'jpeg'

    xmins = [ box[1] ] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [ box[3] ] # List of normalized right x coordinates in bounding box
    # (1 per box)
    ymins = [ box[0] ] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [ box[2] ] # List of normalized bottom y coordinates in bounding box
    # (1 per box)
    classes_text = [ textlabel[newclass].encode('utf-8') ] # List of string class name of bounding box (1 per box)
    classes = [ label[newclass] ] # List of integer class id of bounding box (1 per box)

    tf_record = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf-8')),
        'image/source_id': dataset_util.bytes_feature(filename.encode('utf-8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_record

In [18]:
TRAFFIC_LIGHT = 10
THRESHOLD = 0.8

IMAGE_FILES_DIR = os.path.join(DATA_DIR, 'sim_test_images')
IMAGE_FILES_CSV1 = os.path.join(IMAGE_FILES_DIR, 'session1.csv')
IMAGE_FILES_CSV2 = os.path.join(IMAGE_FILES_DIR, 'session2.csv')

RECORD_FILE_NAME_TRAIN = os.path.join(DATA_DIR, 'train_data', 'train_data.record')
RECORD_FILE_NAME_TEST = os.path.join(DATA_DIR, 'train_data', 'test_data.record')


def load_images_labels(csv_file, images_dir):
    imagefiles = []
    labels = []

    print('IMAGE_FILES_CSV = ', csv_file)

    with open(csv_file) as f:
      cnt = 0
      for line in f:
        if cnt > 0:
          data = line.split(',')
          data_path = data[7].replace("'", "")
          image_path = data_path.split('/')[3]
          imagefiles.append(os.path.join(images_dir, image_path))
          labels.append(int(data[8]))
        cnt += 1
    return imagefiles, labels


imagefiles = []
labels = []

imagefiles1, labels1 = load_images_labels(IMAGE_FILES_CSV1, IMAGE_FILES_DIR)
imagefiles2, labels2 = load_images_labels(IMAGE_FILES_CSV2, IMAGE_FILES_DIR)

print('imagefiles1 len = ', len(imagefiles1))
print('labels1 len = ', len(labels1))

print('imagefiles2 len = ', len(imagefiles2))
print('labels2 len = ', len(labels2))



imagefiles.extend(imagefiles1)
imagefiles.extend(imagefiles2)
labels.extend(labels1)
labels.extend(labels2)

print('imagefiles len = ', len(imagefiles))
print('labels len = ', len(labels))

imagefiles_train, imagefiles_test, labels_train, labels_test = train_test_split(
    imagefiles, labels, test_size=0.15, random_state=42)

print('imagefiles_train len = ', len(imagefiles_train))
print('labels_train len = ', len(labels_train))

print('imagefiles_test len = ', len(imagefiles_test))
print('labels_test len = ', len(labels_test))


def prepare_tf_record_file(images, labels, record_filename):
    print('Preparing tf records: ', record_filename, ' ...')
    writer = tf.python_io.TFRecordWriter(record_filename)
    records_cnt = 0
    with detection_graph.as_default():
      with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path, image_label in zip(images, labels): # TEST_IMAGE_PATHS
          print('processing image ', image_path)
          image = Image.open(image_path)
          # the array based representation of the image will be used later in order to prepare the
          # result image with boxes and labels on it.
          image_np = load_image_into_numpy_array(image)
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
          image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
          (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})

          classes = np.squeeze(classes).astype(np.int32)
          boxes = np.squeeze(boxes)
          scores = np.squeeze(scores)

          light_classes = []
          light_boxes = []
          light_scores = []
          for j in range(len(classes)):
            if classes[j] == TRAFFIC_LIGHT:
                if scores[j] is not None and scores[j] > THRESHOLD:
#                     print("score:", scores[j],"box:",boxes[j])
                    light_classes.append(classes[j])
                    light_boxes.append(boxes[j])
                    light_scores.append(scores[j])
                    record = create_record(boxes[j], image_label, image_np, image_path)
                    writer.write(record.SerializeToString())
                    records_cnt += 1

    writer.close()
    print('recorded {} records'.format(records_cnt))
    
    
prepare_tf_record_file(imagefiles_train, labels_train, RECORD_FILE_NAME_TRAIN)
prepare_tf_record_file(imagefiles_test, labels_test, RECORD_FILE_NAME_TEST)


        
# print('imagefiles = ', imagefiles[:10])
# print('labels = ', labels[:10])

'''
writer = tf.python_io.TFRecordWriter(RECORD_FILE_NAME)
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path, image_label in zip(imagefiles, labels): # TEST_IMAGE_PATHS
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      
      classes = np.squeeze(classes).astype(np.int32)
      boxes = np.squeeze(boxes)
      scores = np.squeeze(scores)
    
      light_classes = []
      light_boxes = []
      light_scores = []
      for j in range(len(classes)):
        if classes[j] == TRAFFIC_LIGHT:
            if scores[j] is not None and scores[j] > THRESHOLD:
                print("score:", scores[j],"box:",boxes[j])
                light_classes.append(classes[j])
                light_boxes.append(boxes[j])
                light_scores.append(scores[j])
                record = create_record(boxes[j], image_label, image_np, image_path)
                writer.write(record.SerializeToString())
#                 print('record = ', record)
    
#       print('boxes = ', boxes)
#       print('classes = ', classes)
#       print('scores = ', scores)
      # Visualization of the results of a detection.
     
#       vis_util.visualize_boxes_and_labels_on_image_array(
#           image_np,
#           np.squeeze(light_boxes),
#           np.squeeze(light_classes).astype(np.int32),
#           np.squeeze(light_scores),
#           category_index,
#           use_normalized_coordinates=True,
#           line_thickness=6)
#       plt.figure(figsize=IMAGE_SIZE)
#       plt.imshow(image_np)
     
        
writer.close()
'''


IMAGE_FILES_CSV =  ../../../../../../data_capstone/sim_test_images/session1.csv
IMAGE_FILES_CSV =  ../../../../../../data_capstone/sim_test_images/session2.csv
imagefiles len =  1100
labels len =  1100
imagefiles_train len =  935
labels_train len =  935
imagefiles_test len =  165
labels_test len =  165
Preparing tf records:  ../../../../../../data_capstone/train_data/train_data.record  ...
processing image  ../../../../../../data_capstone/sim_test_images/session1_136.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_54.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_258.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_286.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_400.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_215.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_321.jpg
processing image  ../../../

processing image  ../../../../../../data_capstone/sim_test_images/session1_128.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_289.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_72.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_76.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_361.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_298.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_284.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_429.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_129.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_439.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_120.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_221.jpg
proces

processing image  ../../../../../../data_capstone/sim_test_images/session1_425.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_533.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_350.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_25.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_5.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_310.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_203.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_104.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_97.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_377.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_277.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_314.jpg
processi

processing image  ../../../../../../data_capstone/sim_test_images/session1_54.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_94.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_43.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_482.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_133.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_167.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_311.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_47.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_311.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_408.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_17.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_180.jpg
processin

processing image  ../../../../../../data_capstone/sim_test_images/session1_395.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_176.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_18.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_474.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_132.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_61.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_272.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_483.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_278.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_153.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_368.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_190.jpg
proces

...

processing image  ../../../../../../data_capstone/sim_test_images/session1_242.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_148.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_288.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_159.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_524.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_35.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_540.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_170.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_104.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_198.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_489.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_455.jpg
proce

processing image  ../../../../../../data_capstone/sim_test_images/session1_205.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_34.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_225.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_508.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_536.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_91.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_347.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_14.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_226.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_241.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_13.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_315.jpg
processi

processing image  ../../../../../../data_capstone/sim_test_images/session1_51.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_173.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_318.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_181.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_488.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_56.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_504.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_296.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_528.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_210.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_513.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_138.jpg
proces

processing image  ../../../../../../data_capstone/sim_test_images/session1_423.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_493.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_72.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_223.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_336.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_462.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_78.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_523.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_412.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_171.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_323.jpg
processing image  ../../../../../../data_capstone/sim_test_images/session1_117.jpg
proces

'\nwriter = tf.python_io.TFRecordWriter(RECORD_FILE_NAME)\nwith detection_graph.as_default():\n  with tf.Session(graph=detection_graph) as sess:\n    # Definite input and output Tensors for detection_graph\n    image_tensor = detection_graph.get_tensor_by_name(\'image_tensor:0\')\n    # Each box represents a part of the image where a particular object was detected.\n    detection_boxes = detection_graph.get_tensor_by_name(\'detection_boxes:0\')\n    # Each score represent how level of confidence for each of the objects.\n    # Score is shown on the result image, together with the class label.\n    detection_scores = detection_graph.get_tensor_by_name(\'detection_scores:0\')\n    detection_classes = detection_graph.get_tensor_by_name(\'detection_classes:0\')\n    num_detections = detection_graph.get_tensor_by_name(\'num_detections:0\')\n    for image_path, image_label in zip(imagefiles, labels): # TEST_IMAGE_PATHS\n      image = Image.open(image_path)\n      # the array based representa